In [1]:
import tensorflow as tf 
import numpy as np
import cv2
from matplotlib import pyplot as plt

import glob
import os

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import imgaug.augmenters as iaa
import imgaug.imgaug
import focal_loss

In [2]:
class ImageDataGen(tf.keras.utils.Sequence):
    def __init__(self, image_list, labels, batch_size=16, input_dim=(512, 512), n_classes=2, shuffle=True, augment=False):
        self.input_dim = input_dim
        self.image_list = image_list
        self.labels = labels
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.augment = augment
        
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.image_list) / self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size: (index + 1) * self.batch_size]
        
        # Find matching image list according to batch size
        list_id_temps = [self.image_list[k] for k in indexes]
        
        # Generate data
        x, y = self.data_generator(list_id_temps)
        if self.augment:
            x = self.augmentor(x)
        
#         x = np.array([img / 255. for img in x])
        return x, y
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_list))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def data_generator(self, list_id_temp):
        # Initialize
        x = np.empty((self.batch_size, self.input_dim[0], self.input_dim[1], 3))
        y = np.empty((self.batch_size), dtype=np.int)
        
        # Generate data
        for index, ids in enumerate(list_id_temp):
            img = cv2.imread(ids)
            img = cv2.resize(img, self.input_dim)
            img = img / 255.
            
            label = self.labels[os.path.basename(os.path.dirname(ids))]
            
            x[index, ] = img
            y[index, ] = label
            
        return x, tf.keras.utils.to_categorical(y, num_classes=self.n_classes)
    
    def augmentor(self, images):
        seq = iaa.Sequential([
            iaa.Fliplr(0.5),
            iaa.Sometimes(0.5,
                iaa.GaussianBlur(sigma=(0, 0.5))
            ),
            iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                rotate=(-25, 25)
            )
        ], random_order=True)
        
        return seq(images=images)

# Build simple CNN

In [3]:
# Hyperparams
input_size = (128, 128, 3)
learning_rate = 1e-4
batch_size = 16
epoch = 50

In [4]:
"""Tensorflow-Keras Implementation of Mish"""

## Import Necessary Modules
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects

class Mish(Activation):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X = Activation('Mish', name="conv1_act")(X_input)
    '''

    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'


def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))

get_custom_objects().update({'Mish': Mish(mish)})

In [5]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.InputLayer(input_shape=input_size))
model.add(tf.keras.layers.Conv2D(64, (3, 3)))
model.add(tf.keras.layers.Activation('Mish'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(64, (3, 3)))
model.add(tf.keras.layers.Activation('Mish'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(128, (3, 3)))
model.add(tf.keras.layers.Activation('Mish'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Activation('Mish'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(2))
model.add(tf.keras.layers.Activation('softmax'))

# Compile model
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 64)      1792      
_________________________________________________________________
activation (Activation)      (None, 126, 126, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        36928     
_________________________________________________________________
activation_1 (Activation)    (None, 61, 61, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 128)       7

# Load data & train

In [6]:
labels = {
    'hot_dog': 1,
    'not_hot_dog': 0
}

train_data = glob.glob('data/train/**/*.jpg', recursive=True)
test_data = glob.glob('data/test/**/*.jpg', recursive=True)

In [7]:
train_gen = ImageDataGen(train_data, labels, batch_size=batch_size, input_dim=input_size[:2], augment=False)
test_gen = ImageDataGen(test_data, labels, batch_size=batch_size, input_dim=input_size[:2], shuffle=False)

In [8]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', mode='max', patience=10, verbose=1, min_delta=0.001
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', mode='max', patience=5, verbose=1, min_delta=0.001
)
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    'weights/hotdog_classifier.h5', monitor='val_accuracy', mode='max', save_best_only=True
)

In [ ]:
history = model.fit(
    train_gen,
    steps_per_epoch=len(train_data) // batch_size,
    epochs=epoch,
    validation_data=test_gen,
    validation_steps=len(test_data) // batch_size,
    callbacks=[reduce_lr, early_stopping, model_ckpt]
)

Epoch 1/50
31/31 [==============================] - 12s 375ms/step - loss: 0.6954 - accuracy: 0.5060 - val_loss: 0.6931 - val_accuracy: 0.5202 - lr: 1.0000e-04
Epoch 2/50
31/31 [==============================] - 12s 381ms/step - loss: 0.6627 - accuracy: 0.5968 - val_loss: 0.6855 - val_accuracy: 0.5343 - lr: 1.0000e-04
Epoch 3/50
31/31 [==============================] - 11s 366ms/step - loss: 0.6482 - accuracy: 0.6290 - val_loss: 0.7076 - val_accuracy: 0.5323 - lr: 1.0000e-04
Epoch 4/50
31/31 [==============================] - 12s 376ms/step - loss: 0.6453 - accuracy: 0.6230 - val_loss: 0.6825 - val_accuracy: 0.5806 - lr: 1.0000e-04
Epoch 5/50
31/31 [==============================] - 12s 390ms/step - loss: 0.6035 - accuracy: 0.7036 - val_loss: 0.6806 - val_accuracy: 0.5645 - lr: 1.0000e-04
Epoch 6/50
31/31 [==============================] - 15s 471ms/step - loss: 0.5699 - accuracy: 0.7319 - val_loss: 0.6944 - val_accuracy: 0.5625 - lr: 1.0000e-04
Epoch 7/50
31/31 [======================

In [ ]:
# plot history
plt.figure(figsize=(30, 10))
plt.subplot(121)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Evaluation

In [ ]:
model = tf.keras.models.load_model('weights/hotdog_classifier.h5', compile=False, custom_objects={'mish': tf.keras.layers.Activation('Mish')})

In [ ]:
y_pred = []
y_true = []
for dat in test_data:
    img = cv2.imread(dat)
    img = cv2.resize(img, input_size[:2])
    img = img / 255.
    y_pred.append(img)
    
    y_true.append(labels[os.path.basename(os.path.dirname(dat))])

y_pred = model.predict(np.array(y_pred))

In [ ]:
true_y_pred = []
for pred in y_pred:
    true_y_pred.append(np.argmax(pred))

In [ ]:
report = classification_report(y_true, true_y_pred)
conf_mat = confusion_matrix(y_true, true_y_pred)

In [ ]:
categories = ['Negative', 'Positive']
group_names = ['True Neg','False Pos', 'False Neg','True Pos']

group_percentage = [
    '{0:.2%}'.format(value) for value in conf_mat.ravel() / conf_mat.sum()
]

cm_labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names,group_percentage)]

cm_labels = np.array(cm_labels).reshape(2, 2)

plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
plt.ylabel("Actual values"   , fontdict = {'size':14}, labelpad = 10)
plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)
sns.heatmap(
    conf_mat, annot=cm_labels, fmt='',
    xticklabels=categories, yticklabels=categories
)

print(report)

# Visually Inspect

In [ ]:
for image_path in test_data[:9]:
    img = cv2.imread(image_path)
    img_copy = cv2.resize(img, input_size[:2])
    img_copy = img_copy / 255.
    
    pred = model.predict(np.expand_dims(img_copy, axis=0))
    
    plt.title(np.argmax(pred))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Convert model to saved model

In [17]:
tf.saved_model.save(model, 'weights/saved_model/')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: weights/saved_model/assets


In [18]:
loaded = tf.saved_model.load('weights/saved_model/')
print(list(loaded.signatures.keys()))  # ["serving_default"]

['serving_default']
